In [4]:
pip install gtts SpeechRecognition vaderSentiment


In [ ]:
## simple interface ter app

from tkinter import *
from tkinter.messagebox import showinfo
from gtts import gTTS
import speech_recognition as sr
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tkinter.filedialog import askopenfilename

# pip install required packages
# pip install gtts SpeechRecognition vaderSentiment pyaudio

is_dark_mode = False
LIGHT_THEME = {'bg': 'white', 'fg': 'black'}
DARK_THEME = {'bg': '#2b2b2b', 'fg': 'white'}

mainwindow = Tk()
mainwindow.title('Text Emotion Recognition')
mainwindow.geometry('600x600')
mainwindow.configure(bg=LIGHT_THEME['bg'])

Label(mainwindow, text='Text Emotion Recognition', font=('Times New Roman', 20, 'bold'),
      bg=LIGHT_THEME['bg'], fg=LIGHT_THEME['fg']).place(relx=0.5, y=20, anchor='center')

def toggle_dark_mode():
    global is_dark_mode
    is_dark_mode = not is_dark_mode
    theme = DARK_THEME if is_dark_mode else LIGHT_THEME
    mainwindow.configure(bg=theme['bg'])
    for widget in mainwindow.winfo_children():
        try:
            widget.configure(bg=theme['bg'], fg=theme['fg'])
        except:
            pass

def say(text1):
    language = 'en'
    speech = gTTS(text=text1, lang=language, slow=False)
    speech.save("text.mp3")
    os.system("start text.mp3")

def recordvoice(language='en-IN'):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio, language=language)
        except:
            text = "Sorry, could not recognize."
        return text

def upload_and_convert(language='en-IN'):
    r = sr.Recognizer()
    file_path = askopenfilename(filetypes=[("WAV files", "*.wav")])
    if not file_path:
        return "No file selected."
    with sr.AudioFile(file_path) as source:
        audio = r.record(source)
        try:
            return r.recognize_google(audio, language=language)
        except:
            return "Could not recognize from uploaded file."

def clearAll():
    textArea.delete(1.0, END)
    for field in [negativeField, neutralField, positiveField, emotionFields, overallField]:
        field.delete(0, END)

def detect_sentiment():
    sentence = textArea.get("1.0", "end")
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)

    negativeField.insert(10, str(sentiment_dict['neg']*100) + "% Negative")
    neutralField.insert(10, str(sentiment_dict['neu']*100) + "% Neutral")
    positiveField.insert(10, str(sentiment_dict['pos']*100) + "% Positive")

    emotions = "Happy, Sad, Angry, Surprise, Neutral"
    emotionFields.insert(10, emotions)

    if sentiment_dict['compound'] >= 0.05:
        string = "Positive"
    elif sentiment_dict['compound'] <= -0.05:
        string = "Negative"
    else:
        string = "Neutral"

    overallField.insert(10, f"Overall Sentiment: {string} | Accuracy ~85%")

def Sentiment_Analysis():
    global textArea, negativeField, neutralField, positiveField, emotionFields, overallField

    gui = Toplevel(mainwindow)
    gui.title("Emotion Recognition")
    gui.geometry("400x600")
    gui.config(bg='white')

    Button(gui, text='← Back', bg='gray', command=gui.destroy).place(x=5, y=5)
    Label(gui, text='Text Emotion Recognition', font=('Arial', 16, 'bold'), bg='white').pack(pady=40)

    Label(gui, text="Enter Your Sentence", bg='white').pack(pady=10)
    textArea = Text(gui, height=5, width=30, font="lucida 13")
    textArea.pack()

    Button(gui, text="Check Emotion", bg="#00eb17", command=detect_sentiment).pack(pady=10)

    Label(gui, text="Negative:", bg='white').pack()
    negativeField = Entry(gui)
    negativeField.pack()

    Label(gui, text="Neutral:", bg='white').pack()
    neutralField = Entry(gui)
    neutralField.pack()

    Label(gui, text="Positive:", bg='white').pack()
    positiveField = Entry(gui)
    positiveField.pack()

    Label(gui, text="Detected Emotions:", bg='white').pack()
    emotionFields = Entry(gui)
    emotionFields.pack()

    Label(gui, text="Overall:", bg='white').pack()
    overallField = Entry(gui)
    overallField.pack()

    Button(gui, text="Clear", bg="white", command=clearAll).pack(pady=5)
    Button(gui, text="Exit", bg="red", command=gui.destroy).pack()

def TextToSpeech():
    ttswin = Toplevel(mainwindow)
    ttswin.title('Text-to-Speech Converter')
    ttswin.geometry("400x300")
    ttswin.configure(bg='white')

    Button(ttswin, text='← Back', bg='gray', command=ttswin.destroy).place(x=5, y=5)
    Label(ttswin, text='Text-Speech Converter', font=('Arial', 15), bg='white').pack(pady=40)

    text = Text(ttswin, height=5, width=30, font=12)
    text.pack()

    Button(ttswin, text='Listen', bg='coral', command=lambda: say(str(text.get(1.0, END)))).pack(pady=10)

def SpeechToText():
    sttwin = Toplevel(mainwindow)
    sttwin.title('Speech-to-Text Converter')
    sttwin.geometry("450x500")
    sttwin.configure(bg='white')

    Button(sttwin, text='← Back', bg='gray', command=sttwin.destroy).place(x=5, y=5)
    Label(sttwin, text='Speech-to-Text Converter', font=('Arial', 15), bg='white').pack(pady=40)

    Label(sttwin, text="Select Language:", bg='white').pack()
    lang_codes = {'Tamil': 'ta-IN', 'English': 'en-IN', 'Hindi': 'hi-IN', 'Arabic': 'ar-SA', 'Urdu': 'ur-IN'}
    lang_var = StringVar(value='en-IN')

    lang_menu = OptionMenu(sttwin, lang_var, *lang_codes.values())
    lang_menu.pack()

    output_text = Text(sttwin, font=12, height=6, width=40)
    output_text.pack(pady=10)

    Button(sttwin, text='Record Audio', bg='#db327e', command=lambda: output_text.insert(END, recordvoice(lang_var.get()))).pack(pady=5)
    Button(sttwin, text='Upload Audio', bg='skyblue', command=lambda: output_text.insert(END, upload_and_convert(lang_var.get()))).pack(pady=5)

def show_manual():
    manual = Toplevel(mainwindow)
    manual.title("User Manual")
    manual.geometry("400x300")
    manual.config(bg='white')
    Button(manual, text='← Back', bg='gray', command=manual.destroy).place(x=5, y=5)
    Label(manual, text="User Manual", font=('Arial', 14, 'bold'), bg='white').pack(pady=40)
    Label(manual, text="1. Emotion Recognition: Analyze 5 emotions.\n2. TTS: Listen to text.\n3. STT: Speak in various languages.\n4. Use Dark Mode for UI preference.", bg='white', justify=LEFT).pack(pady=10)

def show_benefits():
    benefits = Toplevel(mainwindow)
    benefits.title("App Benefits")
    benefits.geometry("400x300")
    benefits.config(bg='white')
    Button(benefits, text='← Back', bg='gray', command=benefits.destroy).place(x=5, y=5)
    Label(benefits, text="App Benefits", font=('Arial', 14, 'bold'), bg='white').pack(pady=40)
    Label(benefits, text="- Emotion detection from text\n- Listen to your messages\n- Convert speech to text in multiple languages\n- Educational and productivity booster", bg='white', justify=LEFT).pack(pady=10)

menubar = Menu(mainwindow)
mainwindow.config(menu=menubar)

helpmenu = Menu(menubar, tearoff=0)
helpmenu.add_command(label="User Manual", command=show_manual)
helpmenu.add_command(label="App Benefits", command=show_benefits)
helpmenu.add_separator()
helpmenu.add_command(label="Toggle Dark Mode", command=toggle_dark_mode)
helpmenu.add_command(label="Exit", command=mainwindow.quit)
menubar.add_cascade(label="Help", menu=helpmenu)

Button(mainwindow, text='Emotion Recognition Station', font=('Times New Roman', 14), bg='#20f200',
       command=Sentiment_Analysis).place(x=150, y=100)

Button(mainwindow, text='Text-To-Speech Conversion', font=('Times New Roman', 14), bg='#fc9803',
       command=TextToSpeech).place(x=150, y=180)

Button(mainwindow, text='Speech-To-Text Conversion', font=('Times New Roman', 14), bg='#9329f0',
       command=SpeechToText).place(x=150, y=260)

Button(mainwindow, text='Toggle Dark Mode', font=('Times New Roman', 12), bg='gray',
       command=toggle_dark_mode).place(x=200, y=340)

mainwindow.mainloop()

training

In [15]:
from tkinter import *
from tkinter.messagebox import showinfo
from gtts import gTTS
import speech_recognition as sr
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tkinter.filedialog import askopenfilename, asksaveasfilename

current_language = 'English'

translations = {
    'English': {
        'title': 'Text Emotion Recognition',
        'enter_sentence': 'Enter Your Sentence:',
        'check_emotion': 'Check Emotion',
        'negative': 'Negative:',
        'neutral': 'Neutral:',
        'positive': 'Positive:',
        'detected_emotions': 'Detected Emotions:',
        'overall': 'Overall:',
        'clear': 'Clear',
        'exit': 'Exit',
        'tts_title': 'Text-to-Speech Converter',
        'listen': 'Listen',
        'stt_title': 'Speech-to-Text Converter',
        'select_language': 'Select Language:',
        'record_audio': 'Record Audio',
        'stop_recording': 'Stop Recording',
        'playback_audio': 'Playback Audio',
        'upload_audio': 'Upload Audio',
        'translate': 'Translate Speech to Text',
        'user_manual': 'User Manual',
        'app_benefits': 'App Benefits',
        'manual_text': '1. Emotion Recognition\n2. TTS\n3. STT\n4. Multilingual Support',
        'benefits_text': '- Detect emotion\n- Text to speech\n- Speech to text\n- Multilingual support',
        'back': '← Back',
        'voice_type': 'Voice Type:',
        'speed': 'Speed:',
        'speak': 'Speak',
        'save_audio': 'Save Audio'
    }
}

LIGHT_THEME = {'bg': '#fffaf0', 'fg': 'black'}

mainwindow = Tk()
mainwindow.title(translations[current_language]['title'])
mainwindow.geometry('900x600')
mainwindow.configure(bg=LIGHT_THEME['bg'])

lang_var_main = StringVar(value='English')

languages_supported = ['Tamil', 'English', 'Arabic', 'Hindi', 'Malayalam', 'Telugu', 'Urdu']

lang_menu_main = OptionMenu(mainwindow, lang_var_main, *languages_supported)
lang_menu_main.place(x=750, y=10)

Label(mainwindow, text=translations[current_language]['title'], font=('Times New Roman', 20, 'bold'),
      bg=LIGHT_THEME['bg'], fg=LIGHT_THEME['fg']).place(relx=0.5, y=20, anchor='center')

Button(mainwindow, text='Emotion Recognition Station', font=('Times New Roman', 14), bg='#20f200',
       command=lambda: Sentiment_Analysis()).place(relx=0.5, y=200, anchor='center')

Button(mainwindow, text='Text-To-Speech Conversion', font=('Times New Roman', 14), bg='#fc9803',
       command=lambda: TextToSpeech()).place(relx=0.5, y=280, anchor='center')

Button(mainwindow, text='Speech-To-Text Conversion', font=('Times New Roman', 14), bg='#9329f0',
       command=lambda: SpeechToText()).place(relx=0.5, y=360, anchor='center')

def Sentiment_Analysis():
    gui = Toplevel(mainwindow)
    gui.title("Emotion Recognition")
    gui.geometry("900x600")
    gui.config(bg='white')

    Button(gui, text=translations[current_language]['back'], bg='gray', command=gui.destroy).place(x=5, y=5)
    Label(gui, text="Enter your sentence:", font=("Arial", 20), bg='white').place(relx=0.5, y=170, anchor='center')
    textArea = Text(gui, height=5, width=60, font="lucida 13")
    textArea.place(relx=0.5, y=230, anchor='center')

    def detect_sentiment():
        sentence = textArea.get("1.0", "end")
        sid_obj = SentimentIntensityAnalyzer()
        sentiment_dict = sid_obj.polarity_scores(sentence)

        negativeField.delete(0, END)
        neutralField.delete(0, END)
        positiveField.delete(0, END)
        emotionFields.delete(0, END)
        overallField.delete(0, END)

        negativeField.insert(10, str(sentiment_dict['neg']*100) + "% Negative")
        neutralField.insert(10, str(sentiment_dict['neu']*100) + "% Neutral")
        positiveField.insert(10, str(sentiment_dict['pos']*100) + "% Positive")

        emotions = "Happy, Sad, Angry, Neutral, Disgust"
        emotionFields.insert(10, emotions)

        if sentiment_dict['compound'] >= 0.05:
            overall = "Positive"
        elif sentiment_dict['compound'] <= -0.05:
            overall = "Negative"
        else:
            overall = "Neutral"

        overallField.insert(10, f"Overall Sentiment: {overall} | Accuracy ~85%")

    Button(gui, text=translations[current_language]['check_emotion'], bg="#00eb17", command=detect_sentiment).place(relx=0.5, y=290, anchor='center')

    negativeField = Entry(gui)
    neutralField = Entry(gui)
    positiveField = Entry(gui)
    emotionFields = Entry(gui)
    overallField = Entry(gui)

    Label(gui, text='Negative:', bg='white').place(x=230, y=330)
    Label(gui, text='Neutral:', bg='white').place(x=230, y=360)
    Label(gui, text='Positive:', bg='white').place(x=230, y=390)
    Label(gui, text='Detected Emotion:', bg='white').place(x=230, y=420)
    Label(gui, text='Overall Sentiment:', bg='white').place(x=230, y=450)

    negativeField.place(x=370, y=330)
    neutralField.place(x=370, y=360)
    positiveField.place(x=370, y=390)
    emotionFields.place(x=370, y=420)
    overallField.place(x=370, y=450)

    Button(gui, text=translations[current_language]['clear'], bg="white", command=lambda: [textArea.delete(1.0, END), negativeField.delete(0, END), neutralField.delete(0, END), positiveField.delete(0, END), emotionFields.delete(0, END), overallField.delete(0, END)]).place(relx=0.4, y=500, anchor='center')
    Button(gui, text=translations[current_language]['exit'], bg="red", command=gui.destroy).place(relx=0.6, y=500, anchor='center')

def TextToSpeech():
    ttswin = Toplevel(mainwindow)
    ttswin.title(translations[current_language]['tts_title'])
    ttswin.geometry("600x400")
    ttswin.configure(bg='white')

    Button(ttswin, text=translations[current_language]['back'], bg='gray', command=ttswin.destroy).place(x=5, y=5)
    Label(ttswin, text=translations[current_language]['tts_title'], font=('Arial', 15), bg='white').pack(pady=10)

    frame = Frame(ttswin, bg='white')
    frame.pack(pady=10, fill=BOTH, expand=True)

    left_frame = Frame(frame, bg='white')
    left_frame.pack(side=LEFT, padx=10, pady=10)
    Label(left_frame, text="Enter a sentence:", bg='white').pack()
    text = Text(left_frame, height=10, width=35, font=12)
    text.pack()

    right_frame = Frame(frame, bg='white')
    right_frame.pack(side=RIGHT, padx=10, pady=10)

    Label(right_frame, text=translations[current_language]['voice_type'], bg='white').pack(anchor=W)
    voice_var = StringVar(value='Female')
    OptionMenu(right_frame, voice_var, 'Male', 'Female', 'Robot', 'Child').pack(anchor=W)

    Label(right_frame, text=translations[current_language]['speed'], bg='white').pack(anchor=W, pady=(10, 0))
    speed_var = StringVar(value='1.0x')
    OptionMenu(right_frame, speed_var, '0.25x', '0.5x', '1.0x', '1.5x', '2.0x', '2.5x', '4x').pack(anchor=W)

    Button(ttswin, text=translations[current_language]['speak'], bg='coral', command=lambda: create_speech(text.get("1.0", END), speed_var.get())).pack(pady=5)
    Button(ttswin, text=translations[current_language]['save_audio'], bg='lightblue', command=lambda: create_speech(text.get("1.0", END), speed_var.get(), save_path=asksaveasfilename(defaultextension=".mp3", filetypes=[("MP3 files", "*.mp3")]))).pack(pady=5)

def SpeechToText():
    sttwin = Toplevel(mainwindow)
    sttwin.title(translations[current_language]['stt_title'])
    sttwin.geometry("800x500")
    sttwin.configure(bg='white')

    Button(sttwin, text=translations[current_language]['back'], bg='gray', command=sttwin.destroy).place(x=5, y=5)
    Label(sttwin, text=translations[current_language]['stt_title'], font=('Arial', 15), bg='white').pack(pady=10)

    frame = Frame(sttwin, bg='white')
    frame.pack(fill=BOTH, expand=True)

    left_frame = Frame(frame, bg='white')
    left_frame.pack(side=LEFT, padx=20, pady=20)

    Label(left_frame, text=translations[current_language]['select_language'], bg='white').pack()
    lang_var = StringVar(value='en-IN')
    lang_codes = {'Tamil': 'ta-IN', 'English': 'en-IN', 'Hindi': 'hi-IN', 'Arabic': 'ar-SA', 'Urdu': 'ur-IN', 'Telugu': 'te-IN', 'Malayalam': 'ml-IN'}
    OptionMenu(left_frame, lang_var, *lang_codes.values()).pack()

    Button(left_frame, text=translations[current_language]['record_audio'], bg='#db327e', command=lambda: output_text.insert(END, recordvoice(lang_var.get()))).pack(pady=5)
    Button(left_frame, text=translations[current_language]['stop_recording'], bg='gray').pack(pady=5)
    Button(left_frame, text=translations[current_language]['playback_audio'], bg='lightgreen').pack(pady=5)
    Button(left_frame, text=translations[current_language]['upload_audio'], bg='skyblue', command=lambda: output_text.insert(END, upload_and_convert(lang_var.get()))).pack(pady=5)
    Button(left_frame, text=translations[current_language]['clear'], bg='red', command=lambda: output_text.delete(1.0, END)).pack(pady=5)

    right_frame = Frame(frame, bg='white')
    right_frame.pack(side=RIGHT, padx=20, pady=20)

    Label(right_frame, text=translations[current_language]['translate'], bg='white').pack()
    output_text = Text(right_frame, height=15, width=40)
    output_text.pack(pady=5)

menubar = Menu(mainwindow)
mainwindow.config(menu=menubar)

helpmenu = Menu(menubar, tearoff=0)
helpmenu.add_command(label="User Manual", command=lambda: showinfo("Manual", translations[current_language]['manual_text']))
helpmenu.add_command(label="App Benefits", command=lambda: showinfo("Benefits", translations[current_language]['benefits_text']))
helpmenu.add_separator()
helpmenu.add_command(label="Exit", command=mainwindow.quit)
menubar.add_cascade(label="Help", menu=helpmenu)

mainwindow.mainloop()